## Portfolio Exercise Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their candidates.   The data for this exercise consists of about 120,000 data points split in 2:1 ratio among training and test files. Each data point includes one column indicating whether or not an individual was sent a promotion for a specific product, and one column indicating whether or not that individual eventually purchased that product. Each individual also had seven additional features associated with them.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user.  Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

Ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group.

* **Net Incremental Revenue (NIR)**

The total number of purchasers that received the promotion times 10 minus the number of promotions given times 0.15 minus the number of purchasers who were not given the promotion times 10.

For a full description of what starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the **promotion_strategy** function to pass to the **test_results** function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers.  

|   | Actual      |    |    | 
|---|-------------|----|----|
| **Predicted**   | Yes| No |  
| Yes             | **I**   | **II**|  
| No              | **III** | **IV**|  

The metrics are only being compared for the individual's we predict should obtain the promotion - that is quadrants I and II here.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equal participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 


Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final test_results function.

In [1]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

# load in the data
train_data = pd.read_csv('training.csv')
test_data = pd.read_csv('Test.csv')

In [ ]:
# Cells for you to work and document as necessary - 
# definitely feel free to add more cells as you need

In [141]:
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


In [9]:
train_data.dtypes

ID             int64
Promotion     object
purchase       int64
V1             int64
V2           float64
V3           float64
V4             int64
V5             int64
V6             int64
V7             int64
dtype: object

In [74]:
train_data.isnull().sum()

ID           0
Promotion    0
purchase     0
V1           0
V2           0
V3           0
V4           0
V5           0
V6           0
V7           0
dtype: int64

In [8]:
train_data.shape

(84534, 10)

In [2]:
train_data['Promotion'].unique()

array(['No', 'Yes'], dtype=object)

In [3]:
train_data['purchase'].unique()

array([0, 1], dtype=int64)

In [4]:
train_data['V1'].unique()

array([2, 3, 0, 1], dtype=int64)

In [7]:
train_data['V2'].unique().shape

(84518,)

In [15]:
train_data.groupby('purchase').describe()

ID                                                        \
            count          mean           std    min       25%      50%   
purchase                                                                  
0         83494.0  62942.917383  36421.387074    1.0  31453.50  62784.5   
1          1040.0  65223.305769  36127.429098  162.0  32540.75  65986.5   

                                  V1            ...   V6            V7  \
              75%       max    count      mean  ...  75%  max    count   
purchase                                        ...                      
0         94406.5  126184.0  83494.0  1.501138  ...  4.0  4.0  83494.0   
1         96326.0  126047.0   1040.0  1.462500  ...  3.0  4.0   1040.0   

                                                       
              mean       std  min  25%  50%  75%  max  
purchase                                               
0         1.701751  0.457492  1.0  1.0  2.0  2.0  2.0  
1         1.697115  0.459727  1.0  1.0  2.0  2.0  2.0  

[2 rows x 64 columns]

In [16]:
from sklearn.linear_model import LogisticRegression

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

## Rebalance the data(好像没有用）

In [76]:
purchased = train_data[train_data['purchase']==1]
non_purchased = train_data[train_data['purchase']==0]

In [77]:
percentage = len(purchased)/float(len(non_purchased))

In [80]:
non_purchased = non_purchased.sample(frac=percentage)

In [82]:
non_purchased.shape, purchased.shape

((1040, 10), (1040, 10))

In [84]:
train_data_balanced = pd.concat([purchased,non_purchased])

In [85]:
train_data_balanced

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
107,162,Yes,1,1,33.030562,-0.732194,2,1,1,1
328,503,Yes,1,3,31.760033,1.518828,2,1,2,2
352,534,Yes,1,2,17.512709,1.259095,2,3,4,2
383,580,Yes,1,1,31.043735,-1.338239,2,1,2,1
423,639,No,1,1,34.901993,-0.905350,2,3,2,2
...,...,...,...,...,...,...,...,...,...,...
33143,49290,No,0,0,22.688070,1.432250,2,3,1,2
81586,121752,Yes,0,1,30.760086,-1.424817,2,4,2,2
68376,101980,No,0,2,27.484254,-0.905350,1,3,2,2
13089,19511,Yes,0,2,29.882796,1.172517,2,2,4,2


In [209]:
X_train = train_data[['V1','V2','V3','V4','V5','V6','V7']]
y_train = train_data['purchase']
X_test = test_data[['V1','V2','V3','V4','V5','V6','V7']]
y_test = test_data['purchase']

In [140]:
y_train[y_train==1]

107      1
328      1
352      1
383      1
423      1
        ..
84300    1
84314    1
84388    1
84425    1
84448    1
Name: purchase, Length: 1040, dtype: int64

## Logistic Regression

In [210]:
LR_model = LogisticRegression(class_weight='balanced')

In [211]:
LR_model.fit(X_train,y_train)

LogisticRegression(class_weight='balanced')

In [212]:
LR_model.score(X_test,y_test)

0.35853541416566626

In [213]:
y_pred = LR_model.predict(X_test)

In [214]:
confusion_matrix(y_pred,y_test)

array([[14548,    95],
       [26622,   385]], dtype=int64)

## RandomForest

In [182]:
clf = RandomForestClassifier(class_weight='balanced')

In [183]:
clf.fit(X_train,y_train)

RandomForestClassifier(class_weight='balanced')

In [184]:
y_pred1 = clf.predict(X_test)

In [188]:
confusion_matrix(y_pred1, y_test)

array([[35618,   392],
       [ 5552,    88]], dtype=int64)

In [106]:
clf.score(X_test,y_test)

0.9877791116446578

In [108]:
clf.classes_

array([0, 1], dtype=int64)

In [112]:
test_data.shape

(41650, 10)

* **Incremental Response Rate (IRR)** 

Ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group.

* **Net Incremental Revenue (NIR)**

The total number of purchasers that received the promotion times 10 minus the number of promotions given times 0.15 minus the number of purchasers who were not given the promotion times 10.

In [206]:
def IRR(confusion_matrix):
    r1 = confusion_matrix[1][1]/(confusion_matrix[1][1] + confusion_matrix[0][1])
    r2 = confusion_matrix[0][1]/(confusion_matrix[0][0] + confusion_matrix[0][1])
    print(r1, r2)
    return r1-r2

In [194]:
def NIR(confusion_matrix):
    return confusion_matrix[1][1] * 10 - (confusion_matrix[1][1] + confusion_matrix[0][1])*0.15 - confusion_matrix[0][1]*10

# A/B test study

## 1) Without promotion

In [191]:
IRR_1 = 0
NIR_1 = 0

## 2) With Promotion

In [207]:
IRR(confusion_matrix(y_pred,y_test))

0.8020833333333334 0.00651890482398957


0.7955644285093438

In [208]:
NIR(confusion_matrix(y_pred,y_test))

2828.0

In [2]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    '''
    promotion = LR_model.predict(df)
    convert = lambda x: 'Yes' if x==1 else 'No'
    vfunc = np.vectorize(convert)
    promotion = vfunc(promotion)    
    '''
    test_data = pd.read_csv('Test.csv')
    promotion = test_data['Promotion']
    #print(promotion)
    
    return promotion

In [3]:
# This will test your results, and provide you back some information 
# on your how well your promotion_strategy will work in practice
from test_results import test_results, score
test_results(promotion_strategy)

339 20748 0 0


C:\Users\BlickWinkel\Desktop\code\BW_Metaverse\DataScience\DSND_Term2-master\portfolio_exercises\Experiments\test_results.py:10: RuntimeWarning: invalid value encountered in longlong_scalars
  irr = n_treat_purch / n_treat - n_ctrl_purch / n_control


In [255]:
test_data = pd.read_csv('Test.csv')
df = test_data[['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7']]

In [243]:
test_data

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,2,No,0,1,41.376390,1.172517,1,1,2,2
1,6,Yes,0,1,25.163598,0.653050,2,2,2,2
2,7,Yes,0,1,26.553778,-1.597972,2,3,4,2
3,10,No,0,2,28.529691,-1.078506,2,3,2,2
4,12,No,0,2,32.378538,0.479895,2,2,1,2
...,...,...,...,...,...,...,...,...,...,...
41645,126166,No,0,1,33.073275,-0.126150,2,2,2,2
41646,126174,Yes,0,0,32.065720,0.999361,2,3,2,1
41647,126176,Yes,1,3,27.691221,1.172517,2,2,1,2
41648,126177,Yes,0,2,32.306427,0.566472,1,2,4,1


In [239]:
df_pred = LR_model.predict(df)

In [242]:
df

,V1,V2,V3,V4,V5,V6,V7
0,1,41.376390,1.172517,1,1,2,2
1,1,25.163598,0.653050,2,2,2,2
2,1,26.553778,-1.597972,2,3,4,2
3,2,28.529691,-1.078506,2,3,2,2
4,2,32.378538,0.479895,2,2,1,2
...,...,...,...,...,...,...,...
41645,1,33.073275,-0.126150,2,2,2,2
41646,0,32.065720,0.999361,2,3,2,1
41647,3,27.691221,1.172517,2,2,1,2
41648,2,32.306427,0.566472,1,2,4,1


In [241]:
df_pred

array([0, 1, 1, ..., 0, 0, 0], dtype=int64)

In [240]:
confusion_matrix(df_pred,df)

ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets